In [1]:
import itertools
import collections
import copy
from functools import cache

In [2]:
# from https://bradfieldcs.com/algos/graphs/dijkstras-algorithm/
import heapq


def calculate_distances(graph, starting_vertex):
    distances = {vertex: float('infinity') for vertex in graph}
    distances[starting_vertex] = 0

    pq = [(0, starting_vertex)]
    while len(pq) > 0:
        current_distance, current_vertex = heapq.heappop(pq)

        # Nodes can get added to the priority queue multiple times. We only
        # process a vertex the first time we remove it from the priority queue.
        if current_distance > distances[current_vertex]:
            continue

        for neighbor, weight in graph[current_vertex].items():
            distance = current_distance + weight

            # Only consider this new path if it's better than any path we've
            # already found.
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                heapq.heappush(pq, (distance, neighbor))

    return distances

## part 1 ##

    e---f---g---h---i---j---k (hallway)
         \ / \ / \ / \ / 
          a1  b1  c1  d1 (rooms)
          |   |   |   |
          a0  bO  cO  dO

Weights are 1 for e-f, j-k, a0-a1, b0-b1, c0-c1, and d0-d1, and 2 for all others.
The weights of 2 make it so we don't have to have rules about not stopping on a hallway 
space above one of the rooms.

In [3]:
graph = {'a0': {'a1': 1},
         'b0': {'b1': 1},
         'c0': {'c1': 1},
         'd0': {'d1': 1},
         'a1': {'a0': 1, 'f': 2, 'g': 2},
         'b1': {'b0': 1, 'g': 2, 'h': 2},
         'c1': {'c0': 1, 'h': 2, 'i': 2},
         'd1': {'d0': 1, 'i': 2, 'j': 2},
         'e': {'f': 1},
         'f': {'e': 1, 'a1': 2, 'g': 2},
         'g': {'f': 2, 'h': 2, 'a1': 2, 'b1': 2},
         'h': {'g': 2, 'i': 2, 'b1': 2, 'c1': 2},
         'i': {'h': 2, 'j': 2, 'c1': 2, 'd1': 2},
         'j': {'i': 2, 'k': 1, 'd1': 2},
         'k': {'j': 1},
        }
nodes = graph.keys()
nodeidx = {val:i for i,val in enumerate(nodes)}
hallway = ['e', 'f', 'g', 'h', 'i', 'j', 'k']
rooms  = [''.join(p) for p in itertools.product('abcd','01')]
move_cost = {'A': 1, 'B': 10, 'C': 100, 'D': 1000}

In [4]:
example_start = {'a1': 'B', 'b1': 'C', 'c1': 'B', 'd1': 'D',
                 'a0': 'A', 'b0': 'D', 'c0': 'C', 'd0': 'A'}
puzzle_start =  {'a1': 'D', 'b1': 'B', 'c1': 'C', 'd1': 'A',
                 'a0': 'C', 'b0': 'A', 'c0': 'D', 'd0': 'B'}
Positions = collections.namedtuple('Positions', nodes)
for node in hallway:
    example_start[node] = None
    puzzle_start[node] = None
example_init = Positions(*[example_start[node] for node in nodes])
puzzle_init = Positions(*[puzzle_start[node] for node in nodes])

In [5]:
example_init

Positions(a0='A', b0='D', c0='C', d0='A', a1='B', b1='C', c1='B', d1='D', e=None, f=None, g=None, h=None, i=None, j=None, k=None)

In [6]:
def is_finished(pos):
    for node in rooms:
        col = node[0]
        val = pos[nodeidx[node]]
        if (val is None) or (val.lower() != col):
            return False
    return True

In [7]:
finished_ex = Positions(*itertools.chain(*itertools.repeat('ABCD', 2), itertools.repeat(None, len(hallway))))
is_finished(finished_ex), is_finished(example_init), is_finished(puzzle_init)

(True, False, False)

In [8]:
def find_valid_rooms(pos):
    valid = []
    for col in 'abcd':
        empty = None
        for row in reversed(range(2)):
            loc = f'{col}{row}'
            val = pos[nodeidx[loc]]
            if val is None:
                empty = row
        if empty == 0:
            valid.append(f'{col}{empty}')
            continue
        if empty is None:
            continue
        if all(pos[nodeidx[f'{col}{row}']].lower() == col for row in reversed(range(empty))):
            valid.append(f'{col}{empty}')
    return valid

In [9]:
valid_test = Positions(a0='B', b0='B', c0='C', d0=None, a1=None, b1='B', c1=None, d1=None, e=None, f=None, g=None, h=None, i=None, j=None, k=None)

In [10]:
find_valid_rooms(valid_test), find_valid_rooms(example_init)

(['c1', 'd0'], [])

In [11]:
@cache
def find_topmost_moveable(pos):
    can_move = []
    for col in 'abcd':
        for row in reversed(range(2)):
            loc = f'{col}{row}'
            val = pos[nodeidx[loc]]
            if val:
                if any(pos[nodeidx[f'{col}{r}']].lower() != col for r in reversed(range(row+1))):
                    can_move.append(loc)
                break
    return can_move

In [12]:
find_topmost_moveable(valid_test), find_topmost_moveable(finished_ex), find_topmost_moveable(example_init)

(['a0'], [], ['a1', 'b1', 'c1', 'd1'])

In [13]:
@cache
def traversal_costs(pos, startnode):
    newgraph = copy.deepcopy(graph)
    for node in graph:
        for endpt in graph[node]:
            if pos[nodeidx[endpt]] is not None:
                newgraph[node][endpt] = float('infinity')
    return calculate_distances(newgraph, startnode)

In [14]:
def allowed_moves(pos, currcost):
    topmost = find_topmost_moveable(pos)
    hallway_occ = [node for node in hallway if pos[nodeidx[node]] is not None]
    # see if anything can move into it's final position
    end_rooms = find_valid_rooms(pos)
    for end_room in end_rooms:
        col = end_room[0]
        for loc in (topmost + hallway_occ):
            val = pos[nodeidx[loc]]
            home_col = val.lower()
            if home_col != col:
                continue
            tcosts = traversal_costs(pos, loc)
            tcost = tcosts[end_room]
            if tcost < float('infinity'):
                # can move to home room
                newpos = list(pos)
                newpos[nodeidx[end_room]] = val
                newpos[nodeidx[loc]] = None
                cost = tcost*move_cost[val] + currcost
                yield Positions(*newpos), cost
                # don't generate any other alternatives, just do this move
                return
    # no moves to home, so generate all possible moves from the rooms into the hallway
    hallway_empty = [node for node in hallway if pos[nodeidx[node]] is None]
    for toprm, hall in itertools.product(topmost, hallway_empty):
        tcosts = traversal_costs(pos, toprm)
        tcost = tcosts[hall]
        if tcost < float('infinity'):
            # can make the move
            val = pos[nodeidx[toprm]]
            newpos = list(pos)
            newpos[nodeidx[hall]] = val
            newpos[nodeidx[toprm]] = None
            cost = tcost*move_cost[val] + currcost
            yield Positions(*newpos), cost

In [15]:
def solve(startpos):
    curr_costs = {startpos: 0}
    curr_positions = [startpos]
    finished_costs = []
    while curr_positions:
        new_positions = set()
        for pos in curr_positions:
            currcost = curr_costs[pos]
            for allowedpos, cost in allowed_moves(pos, currcost):
                if is_finished(allowedpos):
                    finished_costs.append(cost)
                    continue
                if allowedpos in curr_costs:
                    if cost < curr_costs[allowedpos]:
                        curr_costs[allowedpos] = cost
                        new_positions.add(allowedpos)
                else:
                    curr_costs[allowedpos] = cost
                    new_positions.add(allowedpos)
        curr_positions = new_positions
        print(len(curr_positions))
    print('min cost = ', min(finished_costs))

In [16]:
solve(example_init)

28
279
1124
2222
2273
1493
815
256
33
1
0
min cost =  12521


In [17]:
solve(puzzle_init)

28
375
2250
7020
11512
11068
8166
5143
2782
1439
521
75
7
0
min cost =  15338
